In [11]:
import numpy as np
import scipy.fftpack as fourier
import matplotlib
import matplotlib.pyplot as plt
import scipy.io.wavfile as waves
import pyaudio as pa 
import struct 
import statsmodels.api as sm
from scipy.signal import find_peaks

In [13]:
#Definicion de la primera frecuencia de cada nota musical y del array con los nombre de las notas.

hzInicialFa = 21.826            
hzInicialFaS = 23.125           
hzInicialSol = 24.50            
hzInicialSolS = 25.96
hzInicialLa = 27.50
hzInicialLaS = 29.14
hzInicialSi = 30.87
hzInicialDo = 32.7              
hzInicialDoS = 34.65
hzInicialRe = 36.71
hzInicialReS = 38.89
hzInicialMi = 41.2

hzIniciales = np.array([hzInicialFa, hzInicialFaS, hzInicialSol, hzInicialSolS, hzInicialLa, hzInicialLaS, hzInicialSi, hzInicialDo, hzInicialDoS, hzInicialRe, hzInicialReS, hzInicialMi], dtype=float)

nombreNota = np.array(['Fa','Fa#','Sol','Sol#','La','La#','Si','Do','Do#','Re','Re#','Mi'], dtype = str)

In [16]:
#Funciones para conseguir los 0.5 anteriores y los 0.5 seguientes de un Hz

def nAnteriores(Hz):
    vAnteriores = np.array([], dtype=float)
    for i in range(5):
        valor = 0.5 - (i/10)
        vAnteriores = np.append(vAnteriores, (Hz - valor))
    return vAnteriores

def nSiguientes(Hz):
    vSiguientes = np.array([], dtype=float)
    for i in range(5):
        valor = (i/10) + 0.1
        vSiguientes = np.append(vSiguientes, (Hz + valor))
    return vSiguientes

#Funciones para acumular los valores anteiores y siguientes.

def vAnteriores(Hz):
    
    valoresAnteriores = np.array([], dtype=float)
    valoresAnteriores = np.append(valoresAnteriores, nAnteriores(Hz))
    valoresAnteriores = np.append(valoresAnteriores, Hz)
    return valoresAnteriores

def vSiguientes(Hz):
    valoresSiguientes = np.array([], dtype=float)
    valoresSiguientes = np.append(valoresSiguientes, nSiguientes(Hz))
    return valoresSiguientes

#Funcion para el guardado automatico de las frecuencias en un array.

def LlenadoAutomaticoDeHz(Hz):
    allHz = np.array([], dtype=float)
    allHz = np.append(allHz, vAnteriores(Hz))
    allHz = np.append(allHz, vSiguientes(Hz))
    #print(allHz)
    return allHz

In [17]:
#Frecuencias de Fa, Fa# y Sol
FaFaSSolHz = np.array([], dtype=float)
for i in range(3):
    for j in range(9):

        Hz2 = hzIniciales[i] * (2 ** j)
        FaFaSSolHz = np.append(FaFaSSolHz, LlenadoAutomaticoDeHz(Hz2))

#Frecuencias de Sol#, La, La#, Si, Do, Do#, Re, Re#, Mi
allHz = np.array([], dtype=float)
for i in range(3, 12):
    for j in range(8):
        Hz2 = hzIniciales[i] * (2 ** j)
        allHz = np.append(allHz, LlenadoAutomaticoDeHz(Hz2))
